In [1]:
# torch
import torch
from torchvision.transforms import v2 as T
from torchsummary import summary
import webbrowser
import subprocess

# paths
import os
import sys

# set paths
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# set random
import random

# my imports
from datasets.voc_subset import VOCSubset
from models.SoSi_detection import SoSiDetectionModel
from models.trainer import Trainer
from models.model_utils import evaluate_model
from utils.plot_utils import plot_images_from_voc_dataset

# the lifesaver
%load_ext autoreload
%autoreload 2

In [2]:
# tb launch
port = 6006
tb = subprocess.Popen(["tensorboard", "--logdir=runs", f"--port={port}", "--host=localhost"], 
                    stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
webbrowser.open(f"http://localhost:{port}")

True

In [3]:
# torch setup
torch.manual_seed = 42
random.seed = 42
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## ID calculation
Select backbone based on ID sum:

In [4]:
id1 = '308564293'
id2 = '040830861'

id_sum = sum(int(digit) for digit in (id1 + id2))
id_sum = sum(int(digit) for digit in str(id_sum))
print(f'ID digits sum is: {id_sum}, selecting MobilNet V3.')

ID digits sum is: 7, selecting MobilNet V3.


## P1 - Single Class, Single Instance
In this part we will train a detector for a single class, single instance per image.

### Model Construction
We begin by loading the single object, single instance (sosi) detection model, with the MobilNet V3 Backbone.

In [5]:
# build SoSi Model and send to device
model = SoSiDetectionModel()
model.to(device)
# print the summary
summary(model, input_size = (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 112, 112]             432
       BatchNorm2d-2         [-1, 16, 112, 112]              32
         Hardswish-3         [-1, 16, 112, 112]               0
            Conv2d-4         [-1, 16, 112, 112]             144
       BatchNorm2d-5         [-1, 16, 112, 112]              32
              ReLU-6         [-1, 16, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             256
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 64, 112, 112]           1,024
      BatchNorm2d-11         [-1, 64, 112, 112]             128
             ReLU-12         [-1, 64, 112, 112]               0
           Conv2d-13           [-1, 64, 56, 56]             576
      BatchNorm2d-14           [-1, 64,

### Dataset
Next, we load the `subset` dataset with the single class `cat` and split for train and validation. Since we have only ~1000 images we won't split into 3 (test).

In [6]:
# define the selected class
selected_class_name = "cat"

# load the file containing the indices of images with the selected class
indices_file = os.path.join("..","datasets","indices", f"VOC_{selected_class_name}_selected_indices.pth")
class_idx = torch.load(indices_file)

##### Dataset augmentation and train-val split
We have ~1084 instances of the selected class `cat`. To better train the model, we would augment them with additional `k` times the dataset length of random images that are not from this class, to add some negative examples.

The total set of images `aug_idx` will be randomly split to train and valid.

In [7]:
# augment 
k = 2
available_idx = set(range(11540)) - set(class_idx)
aug_idx = random.sample(available_idx, min(k * len(class_idx), len(available_idx))) + class_idx
print(f'Total of {len(aug_idx)} samples, of which {len(class_idx)} contain the selected class {selected_class_name}.')

# split the indices to a train-valid ratio randomly
split_ratio = 0.7
random.shuffle(aug_idx)
train_size = int(split_ratio * len(aug_idx))
train_idx = aug_idx[:train_size] 
val_idx = aug_idx[train_size:] 

print(f'Train indices of length {len(train_idx)} and val indices of length {len(val_idx)}.')

Total of 3252 samples, of which 1084 contain the selected class cat.
Train indices of length 2276 and val indices of length 976.


#### Transforms
Before bulding the dataset we need to define transformations on the data using `get_trainsform`. 

This function has two modes: train = `True` for training and `False` for validation and inference. In train mode, we will add additional agumentations to enhance the training robustness. 

In inference \ validation, only the basic data transformations from the model backbone defaults are executed.

In [ ]:
def get_transform(train, backbone_transforms):
    transforms = []    
    # standard transforms - resizing and center cropping for 1:1 aspect ratio and 224 size
    transforms.append(T.Resize(size = backbone_transforms.resize_size, interpolation = backbone_transforms.interpolation))
    transforms.append(T.CenterCrop(size=backbone_transforms.crop_size))
    
    # if training mode, add flips and jitters
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        transforms.append(T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1))
    
    # standard transforms - normalizing
    transforms.append(T.ToImage())
    transforms.append(T.ToDtype(torch.float32, scale=True)) # scale to 0-1
    # TODO normalization makes everything wierd
    transforms.append(T.Normalize(mean = backbone_transforms.mean, std = backbone_transforms.std))
    
    return T.Compose(transforms)

### Datasets
Build two datasets, each with the appropriate transformations and indices. 

In [9]:
# training dataset
train_dataset = VOCSubset(
    indices_list = train_idx,
    selected_class = selected_class_name,
    single_instance = True,
    transforms = get_transform(train = True, backbone_transforms = model.backbone_transforms()),
    download = False
)

# validation dataset
val_dataset = VOCSubset(
    indices_list = val_idx, 
    selected_class = selected_class_name,
    single_instance = True,
    transforms = get_transform(train = False, backbone_transforms = model.backbone_transforms())
)

print(f'Train dataset of length {len(train_dataset)} and val dataset of length {len(val_dataset)}.')

Train dataset of length 2276 and val dataset of length 976.


#### Test Dataset
We would like to see that the dataset includes these samples and bounding boxes.

In [10]:
plot_images_from_voc_dataset(train_dataset, num_images=4, title="Train Dataset Samples")
plot_images_from_voc_dataset(val_dataset, num_images=4, title="Validation Dataset Samples")

#### Build data loaders:

In [11]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    pin_memory=True,
    drop_last = True
)

val_dataloader= torch.utils.data.DataLoader(
    val_dataset,
    batch_size=256,
    shuffle=False,
    pin_memory=True,
    drop_last = True
)

### Training Preparation

First, we specify the learning hyperparameters.

In [12]:
# learning rate
lr = 0.05
lr_scheduler_step = 10
lr_gamma = 0.5

# epochs
max_epochs = 100

##### Construct Optimizer and Scheduler 

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, lr_scheduler_step, lr_gamma)

##### Construct loss functions
We will use `CrossEntropyLoss` for the classes (with from_logits = `True` as we do not apply softmax in the model), and `MSE` for the bounding box point loss.

In [14]:
boxes_loss_f = torch.nn.SmoothL1Loss() 
class_loss_f = torch.nn.BCEWithLogitsLoss() 
losses = [boxes_loss_f, class_loss_f]

#### Initial evaluation

In [15]:
#TODO Add images every loop, add images before training

### Training loop

In [16]:
trainer = Trainer(device, model, optimizer, train_dataloader, val_dataloader, losses, lr_scheduler = lr_scheduler)

In [17]:
trainer.train(max_epochs)

Training Epoch 35:   0%|          | 0/17 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [18]:
evaluate_model(model, val_dataset, 4 * 4, device=device)

In [19]:
tb.terminate()
torch.save(model.state_dict(), "p1_model_a_0.pth")